In [19]:
import json
import sys

sys.path.append("../")

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from models.doc2vec_model import Doc2VecModel

import pandas as pd
import numpy as np

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
data_path = "../../data/dataset/corpus.jsonl"
max_docs = -1
docs = {}
with open(data_path, "r") as file:
    for line in file:
        data = json.loads(line)
        docs[data["_id"]] = data["text"]

        if max_docs > 0 and len(docs) == max_docs:
            break

print("Number of documents in corpus: {}".format(len(docs)))

vector_size = 20
window = 10
min_count = 60
workers = 16
epochs = 20
d2v = Doc2VecModel(
    docs,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    epochs=epochs,
)

d2v.fit()

d2v.save(
    f"../../models/doc2vec.docs{len(d2v.model.dv)}.vs{vector_size}.win{window}.min{min_count}.ep{epochs}.model"
)

Number of documents in corpus: 1471406


Training: 100%|██████████| 20/20 [27:49<00:00, 83.49s/it]


In [ ]:
query_data_path = "../../data/dataset/queries.jsonl"
raw_queries = {}
with open(query_data_path, "r") as file:
    for line in file:
        data = json.loads(line)
        docs[data["_id"]] = data["text"]

query_ids_df = pd.read_csv("../../data/task1_train.tsv")
queries = {}
for index, row in query_ids_df.iterrows():
    query_id = row["query-id"]

    query_answer_id = row["corpus-id"]
    query_answer_score = row["score"]

    query_text = raw_queries[query_id]

    queries[query_id] = {
        "text": query_text,
        "answer_ids": [(query_answer_id, query_answer_score)],
    }


queries

In [ ]:
TOP_K = 10

for id in queries.keys():
    d2v_query_answers = d2v.find_similar(queries[id]["text"], TOP_K)
    queries[id]["d2v_answer_ids"] = d2v_query_answers
    
    queries[id]["precision@10"] = 0
    queries[id]["recall@10"] = 0

queries